In [ ]:
import time

import graph_gen_integration
from GraphGen.graph_gen_integration import draw_graph

graph_gen = graph_gen_integration.GraphGen([], keyword_reward=1.1)

In [ ]:
graph_gen.reset_model([], 0)

start_time = time.process_time()
print(start_time)

triples = graph_gen.generate() # EMPTY TODO
stop_time = time.process_time()
print(stop_time)

In [ ]:
stop_time-start_time

In [ ]:
graph_gen.free_model()

In [ ]:
# del graph_gen
import gc, torch
gc.collect()
torch.cuda.empty_cache()

In [ ]:
graph_gen_integration.draw_graph(triples)

In [ ]:
print("H")

In [ ]:
len(triples)

## Check accuracy
### Network

In [ ]:
import sys
import logging

nblog = open("acc.log", "a+")
sys.stdout.echo = nblog
sys.stderr.echo = nblog

get_ipython().log.handlers[0].stream = nblog
get_ipython().log.setLevel(logging.INFO)

%autosave 5

In [ ]:
# load testing dataset
import pandas as pd
import networkx as nx
import re
graph_gen_dataset = pd.read_csv('data/webnlg_test.csv', header=0, delimiter='\t')

In [ ]:
def select_k(spectrum, minimum_energy = 0.9):
    running_total = 0.0
    total = sum(spectrum)
    if total == 0.0:
        return len(spectrum)
    for i in range(len(spectrum)):
        running_total += spectrum[i]
        if running_total / total >= minimum_energy:
            return i + 1
    return len(spectrum)

In [ ]:
from graph_gen_integration import Triple
import graph_gen_integration
graph_gen = graph_gen_integration.GraphGen([], keyword_reward=1.1)

In [ ]:
similarities = []
for idx, data in graph_gen_dataset.iterrows():
    if idx >= 300:
        break

    print(f"No. {idx + 1}")
    
    # ground truth
    graph1 = nx.DiGraph()

    _triples = []

    for _triple in data["Triples"].split("<T>"):
        print(_triple)
        try:
            if _triple == "":
                continue
            split = re.split("<R>|<S>", _triple)
            subject = split[0]
            predicate = split[1]
            _object = split[2]

            graph1.add_edge(subject, _object)
            _triples.append(Triple(subject, predicate, _object, 0, 0))

        except Exception as e:
            print(f"NON-STANDARD TRIPLE {_triple} ({e})")
            continue


    print(_triples)

    # Predicted
    graph_gen.reset_model([_triples[0]], 0)
    triples = graph_gen.generate(data["NL"])

    triples.append(_triples[0])

    graph2 = nx.DiGraph()
    
    for triple in triples:
        graph2.add_edge(triple.subject, triple.object)
    
    # Similarity
    laplacian1 = nx.spectrum.laplacian_spectrum(graph1)
    laplacian2 = nx.spectrum.laplacian_spectrum(graph2)

    k1 = select_k(laplacian1)
    k2 = select_k(laplacian2)
    k = min(k1, k2)

    similarity = sum((laplacian1[:k] - laplacian2[:k])**2)
    print(similarity)
    similarities.append(similarity)


In [ ]:
import numpy as np
print(similarities)
print(f"avg: {np.average(similarities)}")

# Gemini (online and really slow arghhh)

In [ ]:
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Extract as much information from the Input as possible, and express the relationships in RDF Triples that compliment the existing RDF triples created from previous text. Do not use information from common sense. Expressed information must be confidently present in the text.
### Example:
(context)
<T>Turkey<R>largestCity<S>Istanbul
(input)
Turkish Nurhan Atasoy was born and raised in the country of Turkey, home of the city of Istanbul and the lira currency.
(output)
<T>Nurhan_Atasoy<R>nationality<S>Turkish_people<T>Turkey<R>currency<S>Turkish_lira<T>Nurhan_Atasoy<R>birthPlace<S>Turkey
### Existing RDF triples:
{}
### Input:
{}"""
import google.generativeai as gemini
import os

gemini.configure(api_key="") # TODO: GEMINI KEY

similarities = []
_max = 100
for idx, data in graph_gen_dataset.iterrows():
    print(f"No. {idx + 1}/{_max}")
    if idx >= 50:
        break
    
    # ground truth
    graph1 = nx.DiGraph()

    _triples = []

    for _triple in data["Triples"].split("<T>"):
        print(_triple)
        try:
            if _triple == "":
                continue
            split = re.split("<R>|<S>", _triple)
            subject = split[0]
            predicate = split[1]
            _object = split[2]

            graph1.add_edge(subject, _object)
            _triples.append(Triple(subject, predicate, _object, 0, 0))

        except Exception as e:
            print(f"NON-STANDARD TRIPLE {_triple} ({e})")
            continue
            
    # Gemini
    model = gemini.GenerativeModel("gemini-1.5-flash")
    response = model.generate_content(prompt.format(_triples[0], data["NL"], ""))
    print(response)

    # print(prompt.format(data["NL"], "").removesuffix("<unused0>"))
    # print(completion.choices[0].message.content)
    graph2 = nx.DiGraph()

    graph2.add_edge(_triples[0].subject, _triples[0].object)

    for _triple in response.text.split("<T>"):
        print(_triple)
        try:
            if _triple == "":
                continue
            split = re.split("<R>|<S>", _triple)
            subject = split[0]
            predicate = split[1]
            _object = split[2]

            graph2.add_edge(subject, _object)


        except Exception as e:
            print(f"NON-STANDARD TRIPLE {_triple} ({e})")
            continue
            
    # Similarity
    laplacian1 = nx.spectrum.laplacian_spectrum(graph1)
    laplacian2 = nx.spectrum.laplacian_spectrum(graph2)

    k1 = select_k(laplacian1)
    k2 = select_k(laplacian2)
    k = min(k1, k2)

    similarity = sum((laplacian1[:k] - laplacian2[:k])**2)
    print(similarity)
    similarities.append(similarity)



In [ ]:
similarities

In [ ]:
import numpy as np
np.median(similarities)

# Gemma (local thingy)

In [ ]:
# Gemma 2b it
import os

from GraphGen.graph_gen_integration import Triple
from unsloth import FastLanguageModel
import torch

class Gemma2bItGen:
    prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Extract as much information from the Input as possible, and express the relationships in RDF Triples that compliment the existing RDF triples created from previous text. Do not use information from common sense. Expressed information must be confidently present in the text.
### Example:
(context)
<T>Turkey<R>largestCity<S>Istanbul
(input)
Turkish Nurhan Atasoy was born and raised in the country of Turkey, home of the city of Istanbul and the lira currency.
(output)
<T>Nurhan_Atasoy<R>nationality<S>Turkish_people<T>Turkey<R>currency<S>Turkish_lira<T>Nurhan_Atasoy<R>birthPlace<S>Turkey
### Existing RDF triples:
{}
### Input:
{}
### Response:
"""

    def __init__(self):
        # init tokenizer, models, etc.
        self.model = None
        self.tokenizer = None

        # self.max_chunk_length = max_chunk_size
        # self.keyword_reward = keyword_reward

        self.setup_model()

    def setup_model(self):
        # setup model helper function
        self.model, self.tokenizer = FastLanguageModel.from_pretrained(
            model_name="../gemma-2-9b-it",
            max_seq_length=2048,
            dtype=None,
            load_in_4bit=True,
        )
        FastLanguageModel.for_inference(self.model)  # Enable native 2x faster inference

    def free_model(self):
        # Free model from GPU memory to load other models
        import gc
        gc.collect()
        torch.cuda.empty_cache()
        del self.model
        del self.tokenizer
        gc.collect()
        torch.cuda.empty_cache()

    def reset_model(self, existing_rdfs):
        pass
    
    def generate(self, _triples, input_text):
        inputs = self.tokenizer(
            [
                self.prompt.format(
                    input_text,
                    _triples,  # output - leave this blank for generation!
                )
            ], return_tensors="pt").to("cuda")
        print(self.prompt.format(
                    input_text,
                    "",  # output - leave this blank for generation!
                ))
        outputs = self.model.generate(
            **inputs,

            # temperature = 0.9,
            # max_new_tokens = max(len(text) + 100, 700),
            # num_beams = 3,
            # early_stopping = True,

            use_cache=True,
            max_new_tokens = 500
            # Use cache = false is broken haha, but beam search is broken when not using cache hahahah ;-;
        )
        response = self.tokenizer.batch_decode(outputs)
        print(response)
        answer_text = response[0].split("### Response:\n")[1]
        graph2 = nx.DiGraph()
        for _triple in answer_text.split("<T>"):
            print(_triple)
            try:
                if _triple == "":
                    continue
                split = re.split("<R>|<S>", _triple)
                subject = split[0]
                predicate = split[1]
                _object = split[2]
    
                graph2.add_edge(subject, _object)
    
            except Exception as e:
                print(f"NON-STANDARD TRIPLE {_triple} ({e})")
                continue
        
        return graph2

In [ ]:
it_gen = Gemma2bItGen()

In [ ]:
# del it_gen
import gc
torch.cuda.empty_cache()
gc.collect()

In [ ]:
import pandas as pd
import networkx as nx
import re
graph_gen_dataset = pd.read_csv('data/webnlg_test.csv', header=0, delimiter='\t')


def select_k(spectrum, minimum_energy = 0.9):
    running_total = 0.0
    total = sum(spectrum)
    if total == 0.0:
        return len(spectrum)
    for i in range(len(spectrum)):
        running_total += spectrum[i]
        if running_total / total >= minimum_energy:
            return i + 1
    return len(spectrum)

In [ ]:
similarities = []
for idx, data in graph_gen_dataset.iterrows():
    print(f"No. {idx+1}")
    if idx >= 500:
        break
    
    # ground truth
    graph1 = nx.DiGraph()

    _t = data["Triples"].split("<T>")[0]
    if _t == "":
        continue
    split = re.split("<R>|<S>", _t)
    subject = split[0]
    predicate = split[1]
    _object = split[2]

    _t_org = f"<T>{subject}<R>{predicate}<S>{_object}"



    for _triple in data["Triples"].split("<T>"):
        print(_triple)
        try:
            if _triple == "":
                continue
            split = re.split("<R>|<S>", _triple)
            subject = split[0]
            predicate = split[1]
            _object = split[2]

            graph1.add_edge(subject, _object)

        except Exception as e:
            print(f"NON-STANDARD TRIPLE {_triple} ({e})")
            continue
            
    # OpenAI
    
    # print(prompt.format(data["NL"], "").removesuffix("<unused0>"))
    graph2 = it_gen.generate(_t_org, data["NL"])
            
    # Similarity
    laplacian1 = nx.spectrum.laplacian_spectrum(graph1)
    laplacian2 = nx.spectrum.laplacian_spectrum(graph2)

    k1 = select_k(laplacian1)
    k2 = select_k(laplacian2)
    k = min(k1, k2)

    similarity = sum((laplacian1[:k] - laplacian2[:k])**2)
    print(similarity)
    similarities.append(similarity)

In [ ]:
print(similarities)

In [ ]:
print("MED")
print(np.median(similarities))
print("MEAN")
print(np.mean(similarities))